# Multilingual Retrivers Pipeline
This notebook:
- Loads multilingual PDF files ***Rich Dad Poor Dad*** (English + Spanish)
- Applies four retrievers strategies (Vector Store Retriever, MMR, Multiquery Retriever, ContextualCompressionRetriever)
- Uses one embedding models (BAAI/bge-m3)
- Stores embeddings into Pinecone indexes

## Install Dependencies

In [1]:
!pip install langchain sentence-transformers pinecone-client python-dotenv langchain-openai langchain-pinecone langchain_community pypdf hf_xet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.3/438.3 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.

## 1. Knowledge documents
### Languages
*   Spanish
*   English

### Knowledge
*   [Padre-rico-padre-pobre-nueva-es](https://drive.google.com/file/d/1Mt8cEOIcXMJykRkknw4zApuNpNRBAo-v/view?usp=drive_link)
*   [Rich-Dad-Poor-Dad-en](https://drive.google.com/file/d/1TEnvsdJvgWFbhmy-UxbCP-5AmdB_hi4D/view?usp=drive_link)





In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Load `.env` File & library

In [3]:
import os
from dotenv import load_dotenv
import glob
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
import pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
load_dotenv("/content/drive/MyDrive/.env")
pinecone_key = os.getenv("PINECONE_API_KEY")
pinecone_index_name = os.getenv("PINECONE_INDEX_NAME")
openai_key = os.getenv("OPENAI_API_KEY")

## 4. Download and Wrap Embedding Models (LaBSE, E5, BGE-M3)

#⚠️⚠️⚠️ Warning before run this code!

#### You need to RUN this code only for one time

In [23]:
# # Need to run this code only for the first time then you don't need to run again and again

# from google.colab import drive
# from sentence_transformers import SentenceTransformer
# import os

# # Step 1: Mount Google Drive
# drive.mount('/content/drive')

# # Step 2: Define model names and target directory
# models = [
#     "sentence-transformers/LaBSE",
#     "intfloat/multilingual-e5-small",
#     "BAAI/bge-m3"
# ]

# save_base_path = "/content/drive/MyDrive/huggingface_models"

# # Step 3: Download and save each model in SentenceTransformer format
# for model_name in models:
#     print(f"Downloading {model_name}...")

#     # Load using SentenceTransformer directly (downloads ST-compatible format)
#     model = SentenceTransformer(model_name)

#     # Format folder name and save path
#     folder_name = model_name.replace("/", "__")
#     save_path = os.path.join(save_base_path, folder_name)
#     os.makedirs(save_path, exist_ok=True)

#     # Save model in SentenceTransformer format
#     model.save(save_path)
#     print(f"Saved to {save_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Saved to /content/drive/MyDrive/huggingface_models/BAAI__bge-m3


## Load the model from drive

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
import os

base_path = "/content/drive/MyDrive/huggingface_models"
models = {
    "sentence-transformers/LaBSE": "sentence-transformers__LaBSE",
    "intfloat/multilingual-e5-small": "intfloat__multilingual-e5-small",
    "BAAI/bge-m3": "BAAI__bge-m3"
}

embeddings = {
    name: HuggingFaceEmbeddings(model_name=os.path.join(base_path, folder))
    for name, folder in models.items()
}


<ipython-input-4-ec9a282940d8>:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  name: HuggingFaceEmbeddings(model_name=os.path.join(base_path, folder))


#Retrievers

In [ ]:
# # Load the existing index for each model
# for model in models:
#     index_name = model.replace("/", "-").lower()
#     embedding_model = embeddings[model]

#     docsearch = PineconeVectorStore.from_existing_index(
#         index_name=index_name,
#         embedding=embedding_model
#     )

#     print(f"Successfully loaded index '{index_name}' for model '{model}'.")


### MMR retrievel

In [7]:
retrievers = {}

for model in models:
    index_name = model.replace("/", "-").lower()
    embedding_model = embeddings[model]

    docsearch = PineconeVectorStore.from_existing_index(
        index_name=index_name,
        embedding=embedding_model
    )

    print(f"Successfully loaded index '{index_name}' for model '{model}'.")

    # Create and store the MMR retriever for this model
    retriever = docsearch.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 3, "lambda_mult": 0.5}
    )

    retrievers[model] = retriever


Successfully loaded index 'sentence-transformers-labse' for model 'sentence-transformers/LaBSE'.
Successfully loaded index 'intfloat-multilingual-e5-small' for model 'intfloat/multilingual-e5-small'.
Successfully loaded index 'baai-bge-m3' for model 'BAAI/bge-m3'.


In [8]:
query = "What is wealth?"

for model, retriever in retrievers.items():
    results = retriever.get_relevant_documents(query)
    print(f"\nResults from model: {model}")
    for i, doc in enumerate(results):
        print(f"Result {i+1}: {doc.page_content}\n")


<ipython-input-8-412ef627ce9e>:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)



Results from model: sentence-transformers/LaBSE
Result 1: —explicó padre rico.
—¿La verdad acerca de qué?, —pregunté.

Result 2: decir, los ricos a largo plazo se enfocan en construir la
columna de activos, antes que nada. Luego, con los ingresos
generados por los activos, compran sus lujos. Los pobres y
la clase media adquieren lujos con su propia sangre y sudor,
y, para colmo, con la herencia de sus hijos.
Piensa que darte un verdadero lujo debe ser una especie
de recompensa por invertir en un activo y desarrollarlo. Por
ejemplo, cuando mi esposa Kim y yo tuvimos algo de dinero

Result 3: lleva el 50 por ciento en impuestos?


Results from model: intfloat/multilingual-e5-small
Result 1: a truly accurate measurement. I could now measure and know where I 
was in terms of my goal to become financially independent.
Although net worth often includes non-cash-producing assets, like 
stuff you bought that now sits in your garage, wealth measures how 
much money your money is making and, th

In [9]:
def call_llm(prompt: str) -> str:
    from openai import OpenAI
    client = OpenAI()

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an expert evaluator."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=50,
        temperature=0.0
    )
    return response.choices[0].message.content.strip()

# Your query
query = "What is wealth?"

scores = {}

for model, retriever in retrievers.items():
    docs = retriever.get_relevant_documents(query)
    context_text = "\n\n".join([f"{i+1}. {doc.page_content}" for i, doc in enumerate(docs)])

    prompt = f"""
You are a helpful evaluator. Please rate from 1 to 10 how well the following retrieved documents answer the question below.
Give your answer as a single integer number only.

Question: "{query}"

Retrieved documents from model '{model}':
{context_text}

Rate the relevance and informativeness of these documents in answering the question.
"""

    print(f"Evaluating model: {model} ...")
    score_text = call_llm(prompt)

    try:
        score = int(score_text)
    except ValueError:
        print(f"Warning: Could not parse score from LLM output: {score_text}")
        score = 0

    scores[model] = score

# Print ranking
print("\nModel evaluation scores:")
for model, score in sorted(scores.items(), key=lambda x: x[1], reverse=True):
    print(f"{model}: {score}")


Evaluating model: sentence-transformers/LaBSE ...
Evaluating model: intfloat/multilingual-e5-small ...
Evaluating model: BAAI/bge-m3 ...

Model evaluation scores:
BAAI/bge-m3: 8
intfloat/multilingual-e5-small: 6
sentence-transformers/LaBSE: 2


### IN THE SUM OF PICK ONE:-


> BAAI/bge-m3 PERFORM WELL IN OUR DATA SO FURTHER WE WILL USE THIS!



## CONNECTION WITH LLM (OPENAI)

In [16]:
embeddings[model]

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Dense({'in_features': 768, 'out_features': 768, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
  (3): Normalize()
), model_name='/content/drive/MyDrive/huggingface_models/sentence-transformers__LaBSE', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [18]:
embeddings["BAAI/bge-m3"]

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='/content/drive/MyDrive/huggingface_models/BAAI__bge-m3', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [22]:
index_name = "baai-bge-m3"
model = "BAAI/bge-m3"

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings[model]
)

print(f"Successfully loaded index '{index_name}' for model '{model}'.")


Successfully loaded index 'baai-bge-m3' for model 'BAAI/bge-m3'.


In [23]:
# Enable MMR in the retriever
retriever = docsearch.as_retriever(
    search_type="mmr",                   # <-- This enables MMR
    search_kwargs={"k": 3, "lambda_mult": 0.5}  # k = top results, lambda_mult = relevance-diversity balance
)

query = "What is wealth?"
mmr_results = retriever.invoke(query)

for i, doc in enumerate(mmr_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Rich Dad Poor Dad
69
It was pretty confusing at first, but after reading it, it began to make 
some sense: 
Wealth is a person’s ability to survive so many number of days 
forward—or, if I stopped working today, how long could I survive?
Unlike net worth—the difference between your assets and liabilities, 
which is often filled with a person’s expensive junk and opinions of what 
things are worth—this definition creates the possibility for developing 
a truly accurate measurement. I could now measure and know where I 
was in terms of my goal to become financially independent.
Although net worth often includes non-cash-producing assets, like 
stuff you bought that now sits in your garage, wealth measures how 
much money your money is making and, therefore, your financial 
survivability.
Wealth is the measure of the cash flow from the asset column 
compared with the expense column.
Let’s use an example. Let’s say I have cash flow from my asset 
column of $1,000 a month.

### Making it as a meaning full output

In [24]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

prompt = PromptTemplate(
    template="""
      You are a helpful financial assistant.
      Answer ONLY from the provided text context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [25]:
question = "what is the wealth?"
retrieved_docs = retriever.invoke(question)

context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

'Rich Dad Poor Dad\n69\nIt was pretty confusing at first, but after reading it, it began to make \nsome sense: \nWealth is a person’s ability to survive so many number of days \nforward—or, if I stopped working today, how long could I survive?\nUnlike net worth—the difference between your assets and liabilities, \nwhich is often filled with a person’s expensive junk and opinions of what \nthings are worth—this definition creates the possibility for developing \na truly accurate measurement. I could now measure and know where I \nwas in terms of my goal to become financially independent.\nAlthough net worth often includes non-cash-producing assets, like \nstuff you bought that now sits in your garage, wealth measures how \nmuch money your money is making and, therefore, your financial \nsurvivability.\nWealth is the measure of the cash flow from the asset column \ncompared with the expense column.\nLet’s use an example. Let’s say I have cash flow from my asset \ncolumn of $1,000 a month

In [26]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

answer = llm.invoke(final_prompt)
print(answer.content)

Wealth is a person’s ability to survive a certain number of days forward without working, measured by the cash flow from assets compared to expenses. It reflects how much money your money is making and your financial survivability.


### English Test

In [38]:
def answer_question(question, retriever, prompt, llm):
    retrieved_docs = retriever.invoke(question)

    context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)

    final_prompt = prompt.invoke({"context": context_text, "question": question})

    answer = llm.invoke(final_prompt)

    return answer.content

questions = [
    "What is the main difference between assets and liabilities according to Rich Dad Poor Dad?",
    "Why does Rich Dad say it's important to have your money work for you?",
    "What role does financial education play in achieving wealth?",
    "How does Rich Dad suggest people handle fear and risk?",
    "What is the significance of cash flow in building wealth?",
    "What is the main city in pakistan"
]

for i, q in enumerate(questions, 1):
    response = answer_question(q, retriever, prompt, llm)
    print(f"Q{i}: {q}")
    print(f"A{i}: {response}")
    print("\n" + "-"*50 + "\n")


Q1: What is the main difference between assets and liabilities according to Rich Dad Poor Dad?
A1: The main difference is that rich people acquire assets, while the poor and middle class acquire liabilities that they think are assets.

--------------------------------------------------

Q2: Why does Rich Dad say it's important to have your money work for you?
A2: Rich Dad says it's important to have your money work for you because it gives you power and control over your financial situation. If you work for money, you give the power to your employer, but if money works for you, you keep the power and can control it.

--------------------------------------------------

Q3: What role does financial education play in achieving wealth?
A3: Financial education plays a crucial role in achieving wealth. It gives individuals power over money and allows them to begin building wealth by understanding how money works. Positive thinking alone is not enough, as many people have not been taught how 

### Spanish Test

In [39]:
def answer_question(question, retriever, prompt, llm):
    retrieved_docs = retriever.invoke(question)

    context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)

    final_prompt = prompt.invoke({"context": context_text, "question": question})

    answer = llm.invoke(final_prompt)

    return answer.content

questions = [
    "¿Cuál es la principal diferencia entre activos y pasivos según Padre Rico Padre Pobre?",
    "¿Por qué dice Padre Rico que es importante hacer que tu dinero trabaje para ti?",
    "¿Qué papel juega la educación financiera en la creación de riqueza?",
    "¿Cómo sugiere Padre Rico que las personas manejen el miedo y el riesgo?",
    "¿Cuál es la importancia del flujo de efectivo en la construcción de riqueza?",
    "¿Cuál es la ciudad principal de Pakistán?"
]

for i, q in enumerate(questions, 1):
    response = answer_question(q, retriever, prompt, llm)
    print(f"Q{i}: {q}")
    print(f"A{i}: {response}")
    print("\n" + "-"*50 + "\n")


Q1: ¿Cuál es la principal diferencia entre activos y pasivos según Padre Rico Padre Pobre?
A1: Según Padre Rico Padre Pobre, la principal diferencia entre activos y pasivos es que los ricos adquieren activos, mientras que los pobres y la clase media adquieren pasivos que consideran activos.

--------------------------------------------------

Q2: ¿Por qué dice Padre Rico que es importante hacer que tu dinero trabaje para ti?
A2: Padre Rico dice que es importante hacer que tu dinero trabaje para ti porque tener más dinero no resolverá los problemas económicos de las personas, ya que la mayoría adquirirá más deudas si se les da más dinero. Además, señala que es más sencillo aprender a trabajar para conseguir dinero, pero si se desea aprender a hacer que el dinero trabaje para uno, él podría enseñar.

--------------------------------------------------

Q3: ¿Qué papel juega la educación financiera en la creación de riqueza?
A3: La educación financiera es crucial en la creación de riqueza, 

# OTHER RETRIVER CODE ARE GIVEN BELOW IN-CASE OF TESTING

In [31]:
index_name = "baai-bge-m3"
model = "BAAI/bge-m3"

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings[model]
)

### Vector store retrievel

In [32]:
# Simple retriever setup
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 2})

results = retriever.invoke("¿Qué es la riqueza?")

for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
el garaje—, la riqueza mide cuánto dinero está produciendo
tu propio dinero y, por tanto, cuál es tu capacidad de
supervivencia ﬁnanciera.
La riqueza es la medida del ﬂujo de efectivo de la
columna de activos, comparada con la de la columna de
gasto.
Usemos un ejemplo. Digamos que en mi columna de
activos tengo un ﬂujo de efectivo de 1000 dólares al mes, y
que mis gastos mensuales ascienden a 2000 dólares.
¿Cuál es mi riqueza?
Volvamos a la deﬁnición de Buckminster Fuller. ¿Cuántos

--- Result 2 ---
acerca de lo que valen las cosas), esta deﬁnición de riqueza
nos da la oportunidad de desarrollar una forma de medición
verdaderamente precisa. En este momento yo podría medir
y saber en dónde me encuentro en relación con mi objetivo
de volverme independiente en el aspecto económico.
A pesar de que el valor neto de una persona a menudo
incluye activos que no producen efectivo —como artículos
que compraste alguna vez y ahora están empolvándose en
el garaje—, la riqueza mide

### Multiquery Retriever

In [33]:
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=docsearch.as_retriever(search_kwargs={"k": 5}),
    llm=ChatOpenAI(model="gpt-3.5-turbo")
)

query = "what is wealth?"
multiquery_results= multiquery_retriever.invoke(query)

for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Rich Dad Poor Dad
69
It was pretty confusing at first, but after reading it, it began to make 
some sense: 
Wealth is a person’s ability to survive so many number of days 
forward—or, if I stopped working today, how long could I survive?
Unlike net worth—the difference between your assets and liabilities, 
which is often filled with a person’s expensive junk and opinions of what 
things are worth—this definition creates the possibility for developing

--- Result 2 ---
Rich Dad Poor Dad
69
It was pretty confusing at first, but after reading it, it began to make 
some sense: 
Wealth is a person’s ability to survive so many number of days 
forward—or, if I stopped working today, how long could I survive?
Unlike net worth—the difference between your assets and liabilities, 
which is often filled with a person’s expensive junk and opinions of what 
things are worth—this definition creates the possibility for developing 
a truly accurate measurement. I could now measure and

### Contextual Compression Retriever

In [36]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document

base_retriever = docsearch.as_retriever(search_kwargs={"k": 5})

llm = ChatOpenAI(model="gpt-3.5-turbo")
compressor = LLMChainExtractor.from_llm(llm)

compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

query = "¿Qué es la riqueza?"
compressed_results = compression_retriever.invoke(query)


for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



--- Result 1 ---
la riqueza mide cuánto dinero está produciendo
tu propio dinero y, por tanto, cuál es tu capacidad de
supervivencia ﬁnanciera.
La riqueza es la medida del ﬂujo de efectivo de la
columna de activos, comparada con la de la columna de
gasto.

--- Result 2 ---
La riqueza es la medida del ﬂujo de efectivo de la columna de activos, comparada con la de la columna de gasto.

--- Result 3 ---
Wealth is a person’s ability to survive so many number of days forward—or, if I stopped working today, how long could I survive?

--- Result 4 ---
La mente es el activo más poderoso que tenemos. Si la entrenamos bien, puede producir enorme riqueza en lo que parecerá solo un instante. Asimismo, una mente no entrenada puede producir el tipo de pobreza extrema que podría destruir a una familia por generaciones.
